<a href="https://colab.research.google.com/github/canerskrc/Deep_Learning_Project/blob/main/GPT_Next_Word_Prediction_with_LSTM_in_Python_A_Dynamic_Text_Completion_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

# 1. Veri Yükleme ve Tokenizasyon
def load_and_tokenize_data(file_path):
    """
    Metin dosyasını yükler ve kelimelere böler.

    Args:
    file_path (str): Yüklenecek dosya yolu.

    Returns:
    list: Tokenize edilmiş kelimeler listesi.
    """
    try:
        with open(file_path, 'r') as file:
            text = file.read().lower()
        print(f'Corpus length: {len(text)}')
        tokenizer = RegexpTokenizer(r'\w+')
        words = tokenizer.tokenize(text)
        return words
    except FileNotFoundError:
        print(f"Error: Dosya {file_path} bulunamadı.")
        return []

# 2. Benzersiz Kelimeleri Bulma ve İndeksleme
def create_unique_word_index(words):
    """
    Benzersiz kelimeler için indeks oluşturur.

    Args:
    words (list): Tokenize edilmiş kelimeler listesi.

    Returns:
    dict: Kelime indeksleri, numpy array: Benzersiz kelimeler
    """
    unique_words = np.unique(words)
    unique_word_index = {word: i for i, word in enumerate(unique_words)}
    return unique_word_index, unique_words

# 3. Eğitim ve Hedef Verilerinin Hazırlanması
def prepare_training_data(words, unique_word_index, word_length):
    """
    Eğitim için giriş ve çıkış verilerini hazırlar (one-hot encoding).

    Args:
    words (list): Tokenize edilmiş kelimeler listesi.
    unique_word_index (dict): Benzersiz kelimelerin indeksleri.
    word_length (int): Her bir örnekteki kelime sayısı (n-gram).

    Returns:
    np.array: Eğitim verisi (X), np.array: Hedef verisi (Y)
    """
    prev_words = []
    next_words = []

    for i in range(len(words) - word_length):
        prev_words.append(words[i:i + word_length])
        next_words.append(words[i + word_length])

    X = np.zeros((len(prev_words), word_length, len(unique_word_index)), dtype=bool)
    Y = np.zeros((len(next_words), len(unique_word_index)), dtype=bool)

    for i, each_words in enumerate(prev_words):
        for j, each_word in enumerate(each_words):
            X[i, j, unique_word_index[each_word]] = 1
        Y[i, unique_word_index[next_words[i]]] = 1

    return X, Y

# 4. Modelin Oluşturulması
def build_model(input_shape, vocab_size):
    """
    LSTM modeli oluşturur.

    Args:
    input_shape (tuple): Giriş verisinin şekli (word_length, vocab_size).
    vocab_size (int): Benzersiz kelime sayısı.

    Returns:
    Sequential: Derlenmiş LSTM modeli.
    """
    model = Sequential()
    model.add(LSTM(128, input_shape=input_shape))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))

    optimizer = RMSprop(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# 5. Modelin Eğitilmesi ve Kaydedilmesi
def train_and_save_model(model, X, Y, batch_size=128, epochs=20, validation_split=0.05):
    """
    Modeli eğitir ve kaydeder.

    Args:
    model (Sequential): LSTM modeli.
    X (np.array): Eğitim verisi.
    Y (np.array): Hedef verisi.
    batch_size (int): Eğitimde kullanılacak batch boyutu.
    epochs (int): Eğitim döngü sayısı.
    validation_split (float): Validation verisi oranı.
    """
    history = model.fit(X, Y, validation_split=validation_split, batch_size=batch_size, epochs=epochs, shuffle=True)
    model.save('keras_next_word_model.h5')
    with open("history.p", "wb") as f:
        pickle.dump(history.history, f)

# 6. Modeli Yükleme ve Geçmişi Görselleştirme
def load_and_plot_history():
    """
    Eğitilen modeli ve geçmişi yükler, ardından doğruluk ve kayıp grafiği çizer.
    """
    model = load_model('keras_next_word_model.h5')
    history = pickle.load(open("history.p", "rb"))

    plt.plot(history['accuracy'], label='train accuracy')
    plt.plot(history['val_accuracy'], label='validation accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

    plt.plot(history['loss'], label='train loss')
    plt.plot(history['val_loss'], label='validation loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

# 7. Metin Tahmini
def sample(preds, top_n=3):
    """
    En yüksek olasılığa sahip tahmin edilen kelimeleri seçer.

    Args:
    preds (np.array): Tahmin edilen olasılıklar.
    top_n (int): Döndürülecek en iyi tahmin sayısı.

    Returns:
    list: En yüksek olasılığa sahip tahminlerin indeksleri.
    """
    preds = np.log(preds + 1e-10)  # Olasılıklarda sıfır olan değerlerden kaçınmak için
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

def predict_completions(text, model, unique_word_index, reverse_word_index, n=3):
    """
    Girilen metin için olası tamamlamaları tahmin eder.

    Args:
    text (str): Girdi metni.
    model (Sequential): Eğitimli LSTM modeli.
    unique_word_index (dict): Benzersiz kelime indeksleri.
    reverse_word_index (dict): İndekslerden kelimelere dönüşüm sözlüğü.
    n (int): Döndürülecek tahmin sayısı.

    Returns:
    list: Tahmin edilen kelime tamamlamaları.
    """
    def prepare_input(text):
        x = np.zeros((1, WORD_LENGTH, len(unique_word_index)))
        for t, word in enumerate(text.split()):
            if word in unique_word_index:
                x[0, t, unique_word_index[word]] = 1
        return x

    x = prepare_input(text.lower())
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [reverse_word_index[idx] for idx in next_indices]

# 8. Tüm Sürecin Çalıştırılması
if __name__ == "__main__":
    WORD_LENGTH = 5  # Bir cümledeki kelime uzunluğu
    path = '1661-0.txt'  # Metin dosyasının yolu

    # Veriyi yükle ve işle
    words = load_and_tokenize_data(path)
    if not words:
        exit()

    unique_word_index, unique_words = create_unique_word_index(words)
    reverse_word_index = {i: word for word, i in unique_word_index.items()}
    X, Y = prepare_training_data(words, unique_word_index, WORD_LENGTH)

    # Modeli oluştur ve eğit
    model = build_model(input_shape=(WORD_LENGTH, len(unique_words)), vocab_size=len(unique_words))
    train_and_save_model(model, X, Y, batch_size=128, epochs=20)

    # Model geçmişini çiz ve tahmin yap
    load_and_plot_history()

    # Örnek metin tamamlama
    test_text = "it is not a lack"
    completions = predict_completions(test_text, model, unique_word_index, reverse_word_index, n=3)
    print(f'Girilen metin için olası tamamlamalar: {completions}')


KeyboardInterrupt: 